In [ ]:
import urllib.request, zipfile
import pandas as pd, numpy as np,xarray as xr
from pathlib import Path
import re, yaml, copy, json
import helper, config_adapter
from helper import RenderJSON

In [ ]:
import itables
itables.init_notebook_mode(all_interactive=True )
itables.options.maxBytes = "1MB"
itables.options.lengthMenu = [25, 10, 50, 100, 200]
itables.options.buttons = ["copyHtml5", "csvHtml5", "excelHtml5"]
itables.options.layout={"topEnd": "pageLength", "top1": "searchBuilder"}

In [ ]:
params = yaml.safe_load(Path("params.yaml").open("r"))
RenderJSON(params)

In [ ]:
zip_path = Path("koe.zip")
zip_path.parent.mkdir(exist_ok=True, parents=True)
urllib.request.urlretrieve(params["koe_link"], zip_path)
zip = zipfile.Path(zip_path)
display(RenderJSON([str(s) for s in zip.iterdir()]))


In [ ]:
timestamps = zip / "songinfo.json"
with timestamps.open("r") as f:
    timestamps = json.load(f)
all = []
for f, k in timestamps.items():
    all.append(pd.DataFrame(k[1], columns=["id", "start", "end", "?1", "?2", "?3", "?id2"]).assign(file=f)[["id", "file", "start", "end"]])
timestamps = pd.concat(all)
timestamps[["start", "end"]] = timestamps[["start", "end"]] /1000
timestamps

In [ ]:


labels = zip / "segment.extraattrvalue.json"
with labels.open("r") as f:
    labels = json.load(f)
labels = pd.DataFrame(labels, columns=["id", "?", "label"])[["id", "label"]]
labels


In [ ]:

all = pd.merge(timestamps, labels, on="id", how="outer").drop(columns="id")
all

In [ ]:
name_template = params["annot_filename_template"]
files = []
for p, grp in all.groupby("file"):
    filename=Path(name_template.format(filestem=Path(p).stem))
    grp.rename(columns=dict(label="name", start="start_seconds", end="stop_seconds"))[["name", "start_seconds", "stop_seconds"]].reset_index(drop=True).to_csv(filename, index=False)
    files.append(dict(audio_filename=p, annotation_filename = filename))
files = pd.DataFrame(files)
files.to_csv(params["file_table_filename"], index=False, sep="\t")
files